In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from collections import defaultdict

from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.dummy import *

# 1.3 Regressors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.dummy import DummyRegressor
import seaborn as sns
import pickle
from scipy.stats import spearmanr
from sklearn.inspection import permutation_importance
import time
from sklearn.impute import SimpleImputer, KNNImputer

import shap
import seaborn as sns

shap.initjs()

# START HERE to Load saved data

In [ ]:
picklepath = 'C:\\Users\\qdickinson\\Jupyter Notebooks\\yeast_multiomics_impute_SHAP\\pickles\\'

pro_train = pd.read_pickle(picklepath+'pro_train.pickle')
#lip_train = pd.read_pickle(picklepath+'lip_train.pickle')
met_train = pd.read_pickle(picklepath+'met_train.pickle')
#ml_train = pd.read_pickle('ml_train.pickle')

pro_val = pd.read_pickle(picklepath+'pro_val.pickle')
#lip_val = pd.read_pickle(picklepath+'lip_val.pickle')
met_val = pd.read_pickle(picklepath+'met_val.pickle')
#ml_val = pd.read_pickle('ml_val.pickle')

pro_test = pd.read_pickle(picklepath+'pro_test.pickle')
#lip_test = pd.read_pickle(picklepath+'lip_test.pickle')
met_test = pd.read_pickle(picklepath+'met_test.pickle')
#ml_test = pd.read_pickle('ml_test.pickle')

pro_train_all = pd.concat([pro_train, pro_val])
#lip_train_all = pd.concat([lip_train, lip_val])
met_train_all = pd.concat([met_train, met_val])
#ml_train_all = pd.read_pickle('ml_train_all.pickle')


pro_train_clean_names = pro_train
pro_train_clean_names.columns = [name.split(' ')[0] for name in pro_train_clean_names.columns.values.tolist()]
pro_test_clean_names = pro_test
pro_test_clean_names.columns = [name.split(' ')[0] for name in pro_test_clean_names.columns.values.tolist()]


#create met_all, remove duplicate glyceric acid

met_all = pd.concat([met_test,met_val,met_train])
column_numbers = [x for x in range(met_all.shape[1])]  # list of columns' integer indices
column_numbers.remove(30) #removing column integer index 0
met_all = met_all.iloc[:, column_numbers] #return all columns except the 0th column

column_names = met_all.columns

#create pro_all

pro_all = pd.concat([pro_test,pro_val,pro_train])

pro_all_clean_names = pro_all
pro_all_clean_names.columns = [name.split(' ')[0] for name in pro_all_clean_names.columns.values.tolist()]

protein = pro_train_all.T

In [ ]:
def model_eval(model, model_name, XTRAIN, XTEST,YTRAIN, YTEST):
    model.fit(XTRAIN, YTRAIN)
    YPRED = model.predict(XTEST)
    print('Mean squared error: %.4f'
      % mean_squared_error(YTEST, YPRED))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.4f'
      % r2_score(YTEST, YPRED))
    # Plot outputs
    plt.rcParams['figure.figsize'] = 5,5
    #sns.jointplot(YTEST, YPRED,  color='black', kind='reg')
    plt.scatter(YTEST, YPRED, color='blue')
    plt.xlabel('true')
    plt.ylabel('pred')
    plt.subplots_adjust(top=0.9)
    plt.suptitle(model_name, fontsize = 16)
    plt.show()
    return mean_squared_error(YTEST, YPRED), r2_score(YTEST, YPRED)

In [ ]:
## try many models
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
model_dict = {
    'dummy': DummyRegressor(),
    'linear': LinearRegression(),
    'lasso' : Lasso(),
    'elastic': ElasticNet(),
    'ridge': Ridge(),
    'Sup Vec Regr': MultiOutputRegressor(SVR()),
    'M/O adaB': MultiOutputRegressor(AdaBoostRegressor()),
    'M/O gboost': MultiOutputRegressor(GradientBoostingRegressor()) , 
    'ExtraTrees': ExtraTreesRegressor(),
    'RandomForest' : RandomForestRegressor(),
    "XGboost" : XGBRegressor()
}


# Loop through all metabolites - look for rho vs importances

In [ ]:
### for metabolites
tables_dict_met = {}
mse_dict_met = {}
r2_dict_met = {}
met_model_dict = {}
met_permimp_dict={}
for i in range(0, len(met_train_all.columns)):
    print(i)
    print(met_train_all.columns[i])
    save_name = met_test.columns[i].replace('(','').replace(')','').replace(':', '.').replace('/','_')
    etr = ExtraTreesRegressor(n_estimators=500, max_depth=50,random_state=42, n_jobs=-1)
    etr.fit(pro_train_all, np.asarray(met_train_all)[:,i])
    met_model_dict[met_train_all.columns[i]] = etr
    
    ## metrics
    y_pred = etr.predict(pro_test)
    tmp_mse = np.round(mean_squared_error(np.asarray(met_test)[:,i],y_pred), 5)##########
    tmp_r2 = np.round(r2_score(np.asarray(met_test)[:,i], y_pred), 5)###########
    mse_dict_met[met_train_all.columns[i]] = tmp_mse ############
    r2_dict_met[met_train_all.columns[i]] = tmp_r2 ############
    print('r2=', r2_dict_met[met_train_all.columns[i]])

    # plot correlation
    lim = 0.1 + np.abs([y_pred, np.asarray(met_test)[:,i]]).max()
    g = sns.jointplot(np.asarray(met_test)[:,i],y_pred,  xlim=[-lim,lim],
                  ylim=[-lim, lim], kind='reg').set_axis_labels('measured', 'predicted', FONTSIZE=24)
    plt.subplots_adjust(top=0.9)
    g.fig.suptitle(met_test.columns[i] +', MSE='+
                   str(np.round(mean_squared_error(np.asarray(met_test)[:,i],y_pred), 6)), 
                   fontsize=24)
    plt.text(-lim+(lim*0.1), lim-lim*0.2, '$r^2$= '+str(tmp_r2)) ############
    plt.savefig('plots_imp\\pmet\\'+met_test.columns[i]+'_'+str(i)+'.svg')
    plt.savefig('plots_imp\\pmet\\'+met_test.columns[i]+'_'+str(i)+'.png')
    plt.close()
    
    # make table with feature importances
    feature_importances = pd.DataFrame(etr.feature_importances_,
                                       index = protein.T.columns,
                                        columns=['importance'])
    
    #get Permutation Importances
    print('starting permutation importance', i)
    start = time.time()
    met_permimp_dict[met_train_all.columns[i]] = permutation_importance(etr, pro_test, 
                                          met_test.iloc[:,i], n_repeats=5, random_state=42, n_jobs=-1)   
    print('computation done for '+str(met_train_all.columns[i]))
    print('seconds = ', time.time() - start)
    print('=============================')
    feature_importances['perm_imp'] = met_permimp_dict[met_train_all.columns[i]].importances_mean
    
    ## get correlation between all protein profiles and the metabolite
    spearman_rho = []
    for prot_i in range(0, len(pro_val.columns) ):
        tmp_rho, tmp_p = spearmanr(np.asarray(met_train_all)[:,i], np.asarray(pro_train_all)[:,prot_i])
        spearman_rho.append(tmp_rho)
    spearman_rhos = pd.DataFrame(spearman_rho,
                                       index = protein.T.columns,
                                        columns=['rho'])
    feature_importances['rho'] = spearman_rhos
    feature_importances['log10(importance)'] = np.log10(feature_importances['importance'])
    feature_importances.sort_values('rho', ascending=False)
    
    # record table to dictionary, and save as CSV
    tables_dict_met[met_val.columns[i]] = feature_importances
    feature_importances.to_csv('tables_imp\\pmet\\'+save_name+'_num'+str(i)+'.csv')
    
    # plot the correlation rho versus the protein value
    sns.set_style('white')
    g = sns.jointplot(np.log10(feature_importances['importance']), spearman_rho)
    sns.set(font_scale=2)
    plt.subplots_adjust(top=0.9)
    g.fig.suptitle(met_train.columns[i] , fontsize=24)
    g.set_axis_labels('log10(Gini Importance)', 'Spearman rho', fontsize=24)
    plt.savefig('plots_imp\\pmet\\'+ save_name+ '_rhoVsGini_num'+str(i)+'.svg')
    plt.close()
    
    ## plot the correlation rho verus mean perm importance
    sns.set_style('white')
    add_small_num = abs(np.min(feature_importances['perm_imp']))+0.00001
    print(add_small_num)
    g = sns.jointplot(np.log10(feature_importances['perm_imp']+add_small_num), spearman_rho)
    sns.set(font_scale=2)
    plt.subplots_adjust(top=0.9)
    g.fig.suptitle(met_train.columns[i] , fontsize=24)
    g.set_axis_labels('Permutation Importance', 'Spearman rho', fontsize=24)
    plt.savefig('plots_imp/pmet/'+ save_name + '_rhoVsPerm_num'+str(i)+'.svg')
    plt.close()
    
pickle.dump(met_permimp_dict, open('pickle_imp\\ETmet_permimp_dict.pickle', 'wb'))
pickle.dump(tables_dict_met, open('pickle_imp\\ETmet_tables_dict.pickle', 'wb'))
pickle.dump(mse_dict_met, open('pickle_imp\\ETmet_mse_dict.pickle', 'wb'))
pickle.dump(r2_dict_met, open('pickle_imp\\ETmet_r2_dict.pickle', 'wb'))
pickle.dump(met_model_dict, open('pickle_imp\\ETmet_model_dict.pickle', 'wb'))



In [ ]:
#Load Saved Model

met_model_dict = pickle.load(open(picklepath+'ETmet_model_dict_imp.pickle', 'rb'))

In [ ]:
def plot_regression_clean(MODEL_DICT, SAVE_NAME,TESTDF, PLOTNUM, YPRED):
    sns.reset_defaults()
    sns.set_style('white')
    plt.rcParams['figure.figsize'] = 3,3
    y_pred = MODEL_DICT[TESTDF.columns[PLOTNUM]].predict(pro_test)
    #print(TESTDF.iloc[:,PLOTNUM].values)
    tmpdf = pd.DataFrame({'true':TESTDF.iloc[:,PLOTNUM].values, 'pred':y_pred})
    #rint(tmpdf)
    #print(y_pred)
    g = sns.regplot('true', 'pred', data = tmpdf, color='black')
    xmin, xmax = plt.xlim()
    xrange = xmax-xmin
    ymin, ymax = plt.ylim()
    yr = ymax-ymin
    plt.xlim(xmin+(xmin*0.1), xmax+xmax*0.15)
    plt.ylim(ymin+(ymin*0.1), ymax+(ymax*0.15) )
    plt.xlabel('measured')
    plt.ylabel('predicted')
    #g.axes.xaxis.set_label_position("top")
    #g.axes.xaxis.set_ticks_position("top")
    #g.axes.xaxis.set_ticks_position("top")

    plt.text(xmin+xrange*0.05, ymax-yr*0.05, TESTDF.columns[PLOTNUM])
    plt.text(xmin+xrange*0.05, ymax-yr*0.14, '$R^2$= '+str(np.round(r2_dict_lip[TESTDF.columns[PLOTNUM]],3) ))
    plt.text(xmin+xrange*0.05, ymax-yr*0.23, 'MSE= '+str(np.round(mse_dict_lip[TESTDF.columns[PLOTNUM]], 3) ))
    #g.set(yticklabels=[-1,'',0,1,2,3])
    #plt.yticks([-1,0,1,2,3])
    plt.savefig(SAVE_NAME +'_regression.svg', bbox_inches='tight')
    plt.close()

# Now metabolites

In [ ]:
def plot_regression_clean2(MODEL_DICT, SAVE_DIR, SAVE_NAME,TESTDF, PLOTNUM, YPRED):
    sns.reset_defaults()
    sns.set_style('white')
    plt.rcParams['figure.figsize'] = 3,3
    y_pred = MODEL_DICT[SAVE_NAME].predict(pro_test)
    #print(TESTDF.iloc[:,PLOTNUM].values)
    tmpdf = pd.DataFrame({'true':TESTDF.iloc[:,PLOTNUM].values, 'pred':y_pred})
    #rint(tmpdf)
    #print(y_pred)
    g = sns.regplot('true', 'pred', data = tmpdf, color='black')
    xmin, xmax = plt.xlim()
    xrange = xmax-xmin
    ymin, ymax = plt.ylim()
    yr = ymax-ymin
    plt.xlim(xmin+(xmin*0.1), xmax+xmax*0.15)
    plt.ylim(ymin+(ymin*0.1), ymax+(ymax*0.15) )
    plt.xlabel('measured')
    plt.ylabel('predicted')
    #g.axes.xaxis.set_label_position("top")
    #g.axes.xaxis.set_ticks_position("top")
    #g.axes.xaxis.set_ticks_position("top")

    plt.text(xmin+xrange*0.05, ymax-yr*0.05, TESTDF.columns[PLOTNUM])
    plt.text(xmin+xrange*0.05, ymax-yr*0.14, '$R^2$= '+str(np.round(r2_dict_met[SAVE_NAME],3) ))
    plt.text(xmin+xrange*0.05, ymax-yr*0.23, 'MSE= '+str(np.round(mse_dict_met[SAVE_NAME], 3) ))
    #g.set(yticklabels=[-1,'',0,1,2,3])
    #plt.yticks([-1,0,1,2,3])
    plt.savefig(SAVE_DIR+ SAVE_NAME +'_regression.svg', bbox_inches='tight')
    plt.close()

In [ ]:
#SHAP

In [ ]:
#ALL SHAPS

shap_dictall = {}

explainer_dictall = {}
for k in range(len(met_all.columns)):
    print(met_all.columns[k])
    key = met_all.columns[k].replace('(','').replace(')','').replace(':', '.').replace('/','_').replace(' ','-').replace('?','')
    explainer = shap.TreeExplainer(met_model_dict[key])
    explainer_dictall[met_all.columns[k]] = explainer
    shap_values = explainer.shap_values(pro_all)
    shap_dictall[met_all.columns[k]] = shap_values
    
    #print(shap_values)
    # summarize the effects of all the features
    shap.summary_plot(shap_values, pro_all_clean_names, show=False, title=str(met_all.columns[k]) )
    #plt.title(met_test.columns[k])
    plt.savefig('pickle_imp/'+key+'_all.svg', bbox_inches="tight")
    plt.savefig('pickle_imp/'+key+'_all.png', bbox_inches="tight")
    plt.close()
    
pickle.dump(explainer_dictall, open('pickle_imp/shap_pmet_all.pickle', 'wb'))    
pickle.dump(shap_dictall, open('pickle_imp/shap_pmet_all.pickle', 'wb'))
    

In [ ]:
explainer_dictall = pickle.load(open(picklepath + "explainer_pmet_all.pickle", 'rb')) 
shap_dictall = pickle.load(open(picklepath+'shap_pmet_all.pickle', 'rb'))

In [ ]:
shap_values=shap_dictall[met_all.columns[20]]

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, pro_all_clean_names, show=False, title='citrate' )
#plt.savefig('plots2/pmet/shap/shap_summary_citrate.svg')

In [ ]:
shap.force_plot(explainer_dictall[met_all.columns[20]].expected_value, shap_values[20,:], pro_test_clean_names.iloc[20,:], 
                show=True, 
                matplotlib=True,
               text_rotation=90)
#plt.savefig('plots2/pmet/shap/forceplot_'+met_test.columns[k]+str(i)+'.svg')
#plt.savefig('plots2/pmet/shap/forceplot_'+met_test.columns[k]+str(i)+'.png')
plt.close()

In [ ]:
plt.rcParams['figure.figsize'] = 3,3
y_pred = met_model_dict['Citric-acid'].predict(pro_test)
g= sns.regplot(met_test.iloc[:,2], y_pred, color='black')
plt.xlim(-1, 3)
plt.ylim(-1, 3)
plt.xlabel('measured')
plt.ylabel('predicted')
g.axes.xaxis.set_label_position("top")
g.axes.xaxis.set_ticks_position("top")
g.axes.xaxis.set_ticks_position("top")

plt.text(-0.8, 2.5, "Citric Acid")
plt.text(-0.8, 2.1, '$r^2$= '+str(r2_dict_met[met_all.columns[20]]) )
plt.text(-0.8, 1.7, 'MSE= '+str(mse_dict_met[met_all.columns[20]]) )

#g.set(yticklabels=[-1,'',0,1,2,3])
plt.yticks([-1,0,1,2,3])

#plt.savefig('plots2/pmet/clean_citric_acid_regression.svg', bbox_inches='tight')